In [1]:
# Importing
import pandas as pd
import altair as alt
import numpy as np

from IPython.display import Markdown
from IPython.display import display
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

pd.set_option('display.max_columns', None)
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
# Load dataset
dwell_denver = pd.read_csv("dwellings_denver.csv")
dwell_ml = pd.read_csv("dwellings_ml.csv")
dwell_neighborhoods_ml = pd.read_csv("dwellings_neighborhoods_ml.csv")


In [3]:
# Correlation
dwell_ml_corr = dwell_ml.drop(columns=(["parcel"]))
dwell_ml_corr = dwell_ml_corr.corr()

dwell_ml_corr
# numbaths, arcstyle_ONE-STORY, livearea


,abstrprd,livearea,finbsmnt,basement,yrbuilt,totunits,stories,nocars,numbdrm,numbaths,sprice,deduct,netprice,tasp,smonth,syear,condition_AVG,condition_Excel,condition_Fair,condition_Good,condition_VGood,quality_A,quality_B,quality_C,quality_D,quality_X,gartype_Att,gartype_Att/Det,gartype_CP,gartype_Det,gartype_None,gartype_att/CP,gartype_det/CP,arcstyle_BI-LEVEL,arcstyle_CONVERSIONS,arcstyle_END UNIT,arcstyle_MIDDLE UNIT,arcstyle_ONE AND HALF-STORY,arcstyle_ONE-STORY,arcstyle_SPLIT LEVEL,arcstyle_THREE-STORY,arcstyle_TRI-LEVEL,arcstyle_TRI-LEVEL WITH BASEMENT,arcstyle_TWO AND HALF-STORY,arcstyle_TWO-STORY,qualified_Q,qualified_U,status_I,status_V,before1980
abstrprd,1.000000,-0.005975,-0.009185,-0.018987,0.006508,-0.003201,-0.006570,-0.021642,-0.019207,-0.001910,-0.004479,-0.010167,-0.004464,-0.004435,0.009356,-0.010188,0.004151,-0.001512,-0.000302,-0.002444,-0.005496,-0.001164,0.004498,-0.003220,-0.002973,0.001884,-0.011194,-0.004111,-0.002677,-0.013597,0.027033,-0.001171,-0.002757,-0.004149,-0.003415,0.013820,0.026104,-0.010412,-0.015640,-0.001864,-0.001840,-0.005004,-0.005452,-0.003896,-0.006298,-0.001782,0.001782,0.001431,-0.001431,-0.004946
livearea,-0.005975,1.000000,0.327176,0.507423,0.294126,0.108458,0.555886,0.498610,0.611727,0.787472,0.120937,0.004511,0.120923,0.121498,0.002336,0.004441,0.012181,0.068989,-0.006395,-0.029366,0.036534,0.328075,0.335088,-0.456753,-0.118227,0.207589,0.354542,0.011765,-0.017322,-0.056288,-0.314118,-0.006061,-0.013317,0.053105,0.092159,-0.069391,-0.178230,0.049876,-0.355304,0.011211,0.123404,0.023559,0.054421,0.188073,0.514282,0.022890,-0.022890,-0.218573,0.218573,-0.362384
finbsmnt,-0.009185,0.327176,1.000000,0.797050,-0.019738,0.029394,0.042908,0.367735,0.198402,0.493902,-0.025937,0.015719,-0.025958,-0.025546,-0.014140,0.022662,-0.156776,0.057584,-0.004647,0.133652,0.074675,0.150025,0.161012,-0.205610,-0.078152,0.072929,0.146331,0.034520,-0.020136,0.196921,-0.358031,-0.000228,0.019928,-0.063803,0.020540,-0.179038,-0.223038,-0.006291,0.231862,-0.010873,0.031979,-0.076950,0.000438,0.039188,0.108561,0.070016,-0.070016,-0.032685,0.032685,0.050170
basement,-0.018987,0.507423,0.797050,1.000000,0.045415,0.043922,0.175431,0.485584,0.352887,0.560880,-0.013844,0.001685,-0.013845,-0.013452,-0.011977,0.012733,-0.099263,0.045727,-0.006104,0.080338,0.058853,0.164441,0.220017,-0.261540,-0.094325,0.067511,0.260370,0.034498,-0.026926,0.170816,-0.450106,0.000789,0.014438,-0.083156,0.039150,-0.249553,-0.292949,0.020114,0.165403,-0.006515,0.032693,-0.099997,0.015539,0.091023,0.294282,0.057727,-0.057727,-0.134235,0.134235,-0.062536
yrbuilt,0.006508,0.294126,-0.019738,0.045415,1.000000,-0.123270,0.306632,0.098045,0.057770,0.383993,0.174688,-0.055379,0.174756,0.174804,0.014563,-0.023510,0.370565,0.017143,-0.007458,-0.348164,-0.111692,0.143585,0.280254,-0.311326,-0.093775,0.079614,0.440297,-0.023402,-0.038982,-0.429308,-0.014904,-0.009809,-0.054264,0.032233,-0.121318,0.181966,0.180968,-0.226299,-0.418635,0.029741,0.005817,0.018751,0.034370,0.014689,0.267128,-0.013385,0.013385,-0.301602,0.301602,-0.824226
totunits,-0.003201,0.108458,0.029394,0.043922,-0.123270,1.000000,0.113267,-0.001299,0.168468,0.151235,-0.006273,-0.010042,-0.006258,-0.006287,0.004920,-0.000091,-0.021579,-0.002381,-0.000476,0.020611,0.005829,-0.020105,0.015159,-0.001946,-0.009540,-0.005944,-0.050265,-0.005141,0.006718,0.032280,0.015976,0.004814,0.030602,-0.006534,0.712506,-0.031200,-0.028310,0.008841,-0.048139,-0.002936,0.001344,-0.007880,-0.008586,0.032129,-0.023193,-0.014180,0.014180,0.023998,-0.023998,0.057912
stories,-0.006570,0.555886,0.042908,0.175431,0.306632,0.113267,1.000000,0.333332,0.391907,0.600626,0.102936,-0.040828,0.102988,0.103438,0.011828,0.003936,0.090286,0.033705,-0.004876,-0.096777,0.003789,0.128591,0.247224,-0.262419,-0.092444,0.011160,0.283404,-0.026703,-0.013935,-0.015746,-0.272010,-0.012891,-0.013313,-0.066948,0.104037,0.026573,0.023378,0.197589,-0.572511,0.041566,0.111710,-0.080743,-0.087974,0.236566,0.524090,0.013997,

In [4]:
dwell_ml.rename(columns={'arcstyle_ONE-STORY' : 'arcstyle_ONE_STORY'}, inplace=True)

# Compute the correlation
correlation_matrix = dwell_ml[['arcstyle_ONE_STORY', 'before1980']].corr()


correlation_df = correlation_matrix.unstack().reset_index(name='correlation')
correlation_df = correlation_df.rename(columns={'level_0': 'attribute', 'level_1': 'target'})

heatmap1 = alt.Chart(correlation_df).mark_rect().encode(
    x=alt.X('attribute:N', title=None, axis=alt.Axis(labelAngle=-45, labelFontSize=10, titleFontSize=16)),
    y=alt.Y('target:N', title=None, axis=alt.Axis(labelFontSize=10, titleFontSize=16)),
    color=alt.Color('correlation:Q', scale=alt.Scale(scheme='viridis'), title='Correlation'),
    tooltip=['attribute', 'target', alt.Tooltip('correlation:Q')]
).properties(
    title=alt.TitleParams(
        text='Correlation Matrix: "arcstyle_ONE_STORY" vs "before1980"',
        fontSize=10,
        anchor='middle',
        fontWeight='bold'
    ),
       width=100,  # Specify the width of the chart
    height=66  # Specify the height of the chart
)

# Compute the correlation
correlation_matrix = dwell_ml[['numbaths', 'before1980']].corr()


correlation_df = correlation_matrix.unstack().reset_index(name='correlation')
correlation_df = correlation_df.rename(columns={'level_0': 'attribute', 'level_1': 'target'})

heatmap2 = alt.Chart(correlation_df).mark_rect().encode(
    x=alt.X('attribute:N', title=None, axis=alt.Axis(labelAngle=-45, labelFontSize=10, titleFontSize=16)),
    y=alt.Y('target:N', title=None, axis=alt.Axis(labelFontSize=10, titleFontSize=16)),
    color=alt.Color('correlation:Q', scale=alt.Scale(scheme='viridis'), title='Correlation'),
    tooltip=['attribute', 'target', alt.Tooltip('correlation:Q')]
).properties(
    title=alt.TitleParams(
        text='Correlation Matrix: "numbaths" vs "before1980"',
        fontSize=10,
        anchor='middle',
        fontWeight='bold'
    ),
      width=100,  # Specify the width of the chart
    height=66  # Specify the height of the chart
)


# Compute the correlation
correlation_matrix = dwell_ml[['stories', 'before1980']].corr()


correlation_df = correlation_matrix.unstack().reset_index(name='correlation')
correlation_df = correlation_df.rename(columns={'level_0': 'attribute', 'level_1': 'target'})

heatmap3 = alt.Chart(correlation_df).mark_rect().encode(
    x=alt.X('attribute:N', title=None, axis=alt.Axis(labelAngle=-45, labelFontSize=10, titleFontSize=16)),
    y=alt.Y('target:N', title=None, axis=alt.Axis(labelFontSize=10, titleFontSize=16)),
    color=alt.Color('correlation:Q', scale=alt.Scale(scheme='viridis'), title='Correlation'),
    tooltip=['attribute', 'target', alt.Tooltip('correlation:Q')]
).properties(
    title=alt.TitleParams(
        text='Correlation Matrix: "stories" vs "before1980"',
        fontSize=10,
        anchor='middle',
        fontWeight='bold'
    ),
    width=100,  # Specify the width of the chart
    height=66  # Specify the height of the chart
)

final = heatmap1 | heatmap2 | heatmap3
final


alt.HConcatChart(...)